In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
import os


import json
import requests


# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)


from scipy.stats import linregress
#pip install citipy
# Import API key   

from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
#output_data_file = "output_data/cities.csv"



ModuleNotFoundError: No module named 'api_keys'

In [ ]:
cityinfo = pd.read_csv("C:/Users/alisa/GIT/PROJECT-3/data/uscities-airport-lat-lng.csv")
cityinfo.head(10)

In [ ]:
cities = cityinfo['city']


In [ ]:
population = cityinfo['population']

In [ ]:
cities

In [ ]:
population

In [ ]:
len(cities)

In [ ]:
#  https://api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}

#url = "http://api.openweathermap.org/data/2.5/weather?"
url = "https://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

#for city in cities:
# Build query URL
query_url = url + "appid=" + weather_api_key + "&units=" + units +  "&q=" + cities[0]

#query_url

# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()

# Get the temperature from the response
#print(f"The weather API responded with: {weather_json}.")

y = json.dumps(weather_json, indent=4, separators=(". ", " = "))
print(y)

In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

#for city in cities:
# Build query URL
query_url = url + "appid=" + weather_api_key + "&units=" + units +  "&q=" 


city_name = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []

#set a counter
index_counter = 0
set_counter = 1


print("start Data Retrieval")
print("-"*10)

for index, city in enumerate (cities):
    
        response = requests.get(query_url + city).json()
        try:
            city_name.append(response['name'])
            lat.append(response['coord']['lat'])
            lng.append(response['coord']['lon'])
            max_temp.append(response['main']['temp_max'])
            humidity.append(response['main']['humidity'])
            cloudiness.append(response['clouds']['all'])
            wind_speed.append(response['wind']['speed'])
            country.append(response['sys']['country'])
            date.append(response['dt'])
           
            
            if index_counter > 49:
                index_counter = 0
                set_counter = set_counter + 1
    
            else:
                index_counter = index_counter + 1
            
                print(f"Processing Record {index_counter} of Set {set_counter} : {city}") 
  
        except(KeyError, IndexError):
            print("City not found...")

print("-"*10)
print("Data Retrieval Complete")
print("-"*10)

In [ ]:
city_weather_1 = pd.DataFrame({'city': city_name,
                            
                             'Max Temp': max_temp,
                             'Humidity': humidity, 
                             'Cloudiness': cloudiness,
                             'Wind Speed': wind_speed,
                             'Country': country,
                             'Date': date
                             
                            })
city_weather_1.head()

In [ ]:
city_weather_1['Date'] = pd.to_datetime(city_weather_1['Date'], unit='s')
city_weather_1['Date'] = city_weather_1['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
city_weather_1

In [ ]:

city_weather_1.drop_duplicates(subset=['city'])

In [ ]:
city_weather_1.to_csv("data/cities.csv")

In [ ]:

city_weather = pd.merge(cityinfo, city_weather_1, how="left", on=["city", "city"])

In [ ]:
city_weather

In [ ]:
city_weather = city_weather.drop_duplicates(subset=['city'])

In [ ]:
city_weather


In [ ]:
city_weather.to_csv("Clean_data/city_weather.csv")

In [ ]:
gmaps.configure(api_key=g_key)
locations = city_weather[["lat", "lng"]]
humidity = city_weather["Humidity"]
locations

In [ ]:
locations.dropna()

In [ ]:
humidity

In [ ]:
fit_weather_data = city_weather.loc[(city_weather['Wind Speed']<= 20) & (city_weather['Cloudiness'] == 0) & 
                                    (city_weather['Max Temp']<=90) & (city_weather['Max Temp'] >= 70)]
fit_weather_data.dropna()
fit_weather_data.head()

In [ ]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
hotel_df = fit_weather_data.loc[:,:]
hotel_df['Hotel Name'] = ""

target_type = "hotel"
radius = 5000

# rewrite params dict

params = {
    "keyword" : "hotel",
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    city_name = row["city"]

    params["location"] = f"{row['lat']}, {row['lng']}"
    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
   #  time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [ ]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure

In [ ]:
http://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={API key}